# Projects

> Create and load pre-built aimagic notebooks for contexts

In [ ]:
#| default_exp projects

In [ ]:
#| export
import json
from pathlib import Path
import ContextKit.contexts as ctx
from fastcore.all import bind

from fastcore.all import call_parse
from typing import Annotated
import sys

In [ ]:
#| export
def create_cell(cell_type: str, outputs: list, source: list):
    assert cell_type in ('code','markdown')
    # source must be split on newline - check execnb
    cell = {'cell_type': cell_type, 'execution_count': None, 'metadata': {}, 'source': source} 
    if cell_type == 'code': cell['outputs'] = outputs
    return cell

In [ ]:
#| export
def create_magic_template_cells(ctx_name):
    _fn = bind(create_cell,outputs=[], cell_type='code')  
    _ctx_dict = getattr(globals()['ctx'], templates[ctx_name], None)()
    cells = [_fn(source=[f"from ContextKit.contexts import *"]),
            _fn(source=[f"{templates[ctx_name]} = {templates[ctx_name]}()\n"] + [f"{k} = {templates[ctx_name]}['{k}']\n" for k in _ctx_dict.keys()]),
            _fn(source=[f"%%ai -c\nPlease read {', '.join([f'$`{k}`' for k in _ctx_dict.keys()])} and give a brief description of what each includes."]),]
    return cells

In [ ]:
#| export
def create_nb(cells, path):
    assert isinstance(cells, list)
    nb = {"cells": cells, "metadata": {}, "nbformat": 4, "nbformat_minor": 4}
    Path(path).write_text(json.dumps(nb, indent=2))

In [ ]:
#| export
def create_magic_template_nb(ctx_name, path):
    create_nb(create_magic_template_cells(ctx_name), path=path)

In [ ]:
#| export
templates = {name[4:]: name for name in dir(ctx) if name.startswith('ctx_')}

In [ ]:
create_magic_template_nb('fasthtml','../testnb2.ipynb')

dict_keys(['fasthtml_llms-ctx', 'hypermedia_summary'])


In [ ]:
#|export
@call_parse
def ck(ctx_name: Annotated[str, "Context name (one of: " + ", ".join(templates.keys()) + ")"],
       path: Annotated[str, "Output path for the notebook"]):
    """Create a magic template notebook for a chosen context."""
    if ctx_name not in templates:
        print(f"Error: {ctx_name} is not a valid context name.")
        print(f"Valid options are: {', '.join(templates.keys())}")
        sys.exit(1)
    create_magic_template_nb(ctx_name, path)
    print(f"Notebook created at {path}")

# To make this CLI available when the library is installed, add this to setup.py: